<a href="https://colab.research.google.com/github/jordanmsouza/Hacka_Pos_Fiap/blob/main/TreinamentoModeloHacka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Baixando dependências

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

## Treinando o modelo

In [ ]:
from ultralytics import YOLO
import shutil
import os

# Caminho do Google Drive para salvar o modelo
save_path = "/content/drive/MyDrive/HackaFiap/Dataset/NovoDataset"
os.makedirs(save_path, exist_ok=True)  # Garante que o diretório existe

# Função para treinar o modelo YOLOv8
def train_model():
    #model = YOLO('yolov8n.pt')  # Carregar modelo pré-treinado YOLOv8
    #model.train(data='/content/dataset.yaml', epochs=50, imgsz=640)  # Especificar dataset e parâmetros

    # Caminho da pasta de resultados
    runs_path = "/content/runs"

    # Copiar a pasta runs para o Google Drive
    if os.path.exists(runs_path):
        shutil.copytree(runs_path, os.path.join(save_path, "runs"), dirs_exist_ok=True)
        print(f"Pasta 'runs' salva em: {save_path}/runs")
    else:
        print("Erro: Pasta 'runs' não encontrada.")

# Script principal
if __name__ == "__main__":
    train_model()



Pasta 'runs' salva em: /content/drive/MyDrive/HackaFiap/Dataset/NovoDataset/runs


In [ ]:
import os
import shutil

# Caminhos das pastas originais e novas
base_path = "/content/drive/MyDrive/HackaFiap/Dataset/NovoDataset"
label_folders = {
    "train": os.path.join(base_path, "labels/train"),
    "test": os.path.join(base_path, "labels/test")
}
image_folders = {
    "train": os.path.join(base_path, "images/train"),
    "test": os.path.join(base_path, "images/test")
}
new_label_folders = {
    "train": os.path.join(base_path, "labels/train_renamed"),
    "test": os.path.join(base_path, "labels/test_renamed")
}
new_image_folders = {
    "train": os.path.join(base_path, "images/train_renamed"),
    "test": os.path.join(base_path, "images/test_renamed")
}

# Classes do dataset.yaml
class_names = ['knife', 'pistol', 'fork', 'spoon','person']

# Função para criar pastas se não existirem
def create_folders(folders):
    for folder in folders.values():
        if not os.path.exists(folder):
            os.makedirs(folder)

# Função para renomear arquivos e criar pastas correspondentes
def rename_files(label_folder, image_folder, new_label_folder, new_image_folder):
    # Contadores para cada classe
    class_counters = {class_name: 1 for class_name in class_names}

    for label_file in os.listdir(label_folder):
        if not label_file.endswith(".txt"):
            continue  # Ignorar arquivos que não sejam .txt

        label_path = os.path.join(label_folder, label_file)
        image_name = label_file.replace(".txt", ".jpg")  # Assumindo que as imagens têm extensão .jpg
        image_path = os.path.join(image_folder, image_name)

        # Verificar se a imagem correspondente existe
        if not os.path.exists(image_path):
            print(f"Imagem correspondente não encontrada para {label_file}. Ignorando...")
            continue

        with open(label_path, "r") as file:
            lines = file.readlines()
            if not lines:
                print(f"Rótulo vazio encontrado em {label_file}. Ignorando...")
                continue

            # Usar a primeira linha do rótulo para determinar o índice da classe
            first_line = lines[0].strip().split()
            class_index = int(first_line[0])

            if class_index < 0 or class_index >= len(class_names):
                print(f"Índice de classe inválido em {label_file}: {class_index}. Ignorando...")
                continue

            # Nomear o novo arquivo de acordo com a classe
            class_name = class_names[class_index]
            new_base_name = f"{class_name}{class_counters[class_name]}"
            class_counters[class_name] += 1

            # Caminhos para os novos arquivos
            new_label_path = os.path.join(new_label_folder, f"{new_base_name}.txt")

            # Tratar imagem com múltiplos pontos no nome
            # Vamos garantir que a extensão correta (".jpg") seja aplicada, mesmo se houver pontos adicionais no nome
            image_name_without_ext, ext = os.path.splitext(image_name)
            new_image_name = f"{new_base_name}{ext}"  # Garantir que a extensão original (.jpg) seja mantida
            new_image_path = os.path.join(new_image_folder, new_image_name)

            # Renomear o arquivo de label primeiro
            shutil.copy(label_path, new_label_path)
            print(f"Renomeado label: {label_file} -> {new_base_name}.txt")

            # Agora renomear a imagem correspondente
            shutil.copy(image_path, new_image_path)
            print(f"Renomeada imagem: {image_name} -> {new_image_name}")

# Criar as novas pastas
create_folders(new_label_folders)
create_folders(new_image_folders)

# Processar e renomear os arquivos de treino e teste
rename_files(
    label_folder=label_folders["train"],
    image_folder=image_folders["train"],
    new_label_folder=new_label_folders["train"],
    new_image_folder=new_image_folders["train"]
)
rename_files(
    label_folder=label_folders["test"],
    image_folder=image_folders["test"],
    new_label_folder=new_label_folders["test"],
    new_image_folder=new_image_folders["test"]
)

print("Renomeação concluída!")


In [ ]:
import os

# Caminhos das pastas
image_dir = "/content/drive/MyDrive/HackaFiap/Dataset/NovoDataset/images/test_renamed"
label_dir = "/content/drive/MyDrive/HackaFiap/Dataset/NovoDataset/labels/test_renamed"

# Criar um conjunto com os nomes dos arquivos de rótulo (sem extensão)
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')}

# Percorrer a pasta de imagens
for image_file in os.listdir(image_dir):
    image_name, ext = os.path.splitext(image_file)

    # Verifica se a imagem tem um rótulo correspondente
    if image_name not in label_files:
        os.remove(os.path.join(image_dir, image_file))
        print(f"Removendo imagem sem rótulo: {image_file}")

print("✅ Apenas imagens com rótulos foram mantidas!")


✅ Apenas imagens com rótulos foram mantidas!
